# 載入資料

In [1]:
# 如果有使用 coloab 再執行此 cell
from google.colab import drive
drive.mount('/gdrive')
!ln -s /gdrive/MyDrive/Credit_Card/ /content/

Mounted at /gdrive


In [ ]:
f1id = '1SvJ_IiHr-ndJDG_sBf6NCn0lMKUxPIlf'
f2id = '1lZPv46zul32Xbr1qHES66YRzMa-A7MzB'


!pip3 install gdown
import gdown
url = 'https://drive.google.com/uc?id=%s'%(f1id)
output = 'train.csv'
gdown.download(url, output, quiet=False)
url = 'https://drive.google.com/uc?id=%s'%(f2id)
output = 'test.csv'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1SvJ_IiHr-ndJDG_sBf6NCn0lMKUxPIlf
To: /content/train.csv
132MB [00:01, 90.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1lZPv46zul32Xbr1qHES66YRzMa-A7MzB
To: /content/test.csv
36.2MB [00:00, 93.1MB/s]


'test.csv'

In [1]:
import pandas as pd
import numpy as np

# Load Data and Overview

In [2]:
df = pd.read_csv('train.csv').set_index('txkey')
df.shape

(1521787, 22)

In [3]:
y_train = df['fraud_ind']
del df['fraud_ind']

In [4]:
# testing data 的要放進來，這樣 labeling encoding 才不會漏掉
df_test = pd.read_csv('test.csv').set_index('txkey')
df_test.shape

(421665, 21)

In [5]:
df_test.head(2)

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd
txkey,,,,,,,,,,,,,,,,,,,,,
592489,6881,163188,116799,513.8,5,0,N,0,N,N,...,N,0,102,215328.0,457,59360,N,0,102,0
592452,6881,163188,116799,513.8,5,0,N,0,N,N,...,N,0,102,222007.0,457,59360,N,0,102,0


In [6]:
df = pd.concat((df, df_test), 0)
df.shape

(1943452, 21)

In [7]:
df.keys()

Index(['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
       'flbmk', 'flg_3dsmk', 'hcefg', 'insfg', 'iterm', 'locdt', 'loctm',
       'mcc', 'mchno', 'ovrlt', 'scity', 'stocn', 'stscd'],
      dtype='object')

In [8]:
# 欄位類別判定
col_names_cont = [] # 數值型資料
col_names_disc = [] # 類別型資料
col_has_na = [] # 待補NA資料
for c in df.keys():
    uni = df[c].unique()
    n_na = pd.isna(df[c]).sum() # NA 數量
    if n_na > 0:
        col_has_na.append(c)       
    if len(uni) < 200 :
        print(f"{c}: uni={uni}")
        col_names_disc.append(c)
    else:
        info = [ df[c].max(), df[c].min(), df[c].mean(), df[c].std()]
        info = [ round(x,2) for x in info ]
        offset = 1 if n_na else 0
        diversity = (len(uni)-offset)/(len(df)-n_na)
        print(f"{c}: dtype={df[c].dtype}, n_na={n_na}")
        print("       max={}, min={}, mean={}, std={}, diversity={:.2f}%".format(*info, diversity*100 ) )
        if diversity == 1.0:
            print(f"       Delete col [{c}] due to diversity is 100% ")
        else:
            col_names_cont.append(c)

acqic: dtype=int64, n_na=0
       max=6884, min=0, mean=6004.54, std=1505.74, diversity=0.35%
bacno: dtype=int64, n_na=0
       max=163886, min=0, mean=82087.41, std=47323.48, diversity=8.40%
cano: dtype=int64, n_na=0
       max=213575, min=0, mean=109045.28, std=61103.56, diversity=10.93%
conam: dtype=float64, n_na=0
       max=7208.77, min=0.0, mean=651.62, std=403.22, diversity=4.44%
contp: uni=[5 3 6 2 4 0 1]
csmcu: uni=[ 0 62 74 61 13 67 60 26 49 10 71 56 20 38 22  4 47 54 75 32 66 29 28 14
 50 30 31 45 24 40 35 59  3 48 63 52 41 23 17 16 73 51 53 44 18 12 15  5
 70 21 39 27 42 11 72  6 25  7 43  2 19 37  1 64 57 36 68 55  9 46 34 69
  8 33 65 58]
ecfg: uni=['N' 'Y']
etymd: uni=[ 0  2  5  4  8  1  9  6  3  7 10]
flbmk: uni=['N' 'Y' nan]
flg_3dsmk: uni=['N' 'Y' nan]
hcefg: uni=[5 0 3 1 2 7 8 9 6 4]
insfg: uni=['N' 'Y']
iterm: uni=[0 1 2 8 3 4 7 6 5]
locdt: uni=[ 33   9   6   5   7  10   8  11  13  14  22  21  25  24  15  12  19  28
  18  17  16  23  31  26   2  30   4  20  27  29  

- 看起來數值型資料都沒有缺失值(n_na==0)

In [9]:
col_has_na

['flbmk', 'flg_3dsmk']

In [10]:
# 處理缺失值
df = df.fillna("NA")

# Data Preprocessing

## 數值型

In [11]:
df_cont = df[col_names_cont].copy()
df_cont.head()

,acqic,bacno,cano,conam,loctm,mcc,mchno,scity
txkey,,,,,,,,
516056,6881,113261,38038,513.80,172652.0,457,59333,0
4376,0,134508,45725,465.62,105114.0,451,0,5817
483434,6881,15408,188328,513.80,152458.0,457,59333,0
1407164,6716,157159,29967,1016.11,172946.0,247,50436,3281
1051004,5975,105985,81305,713.66,182129.0,263,93775,5817


In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_cont = scaler.fit_transform(df_cont)
X_cont.shape, type(X_cont)

((1943452, 8), numpy.ndarray)

In [13]:
df_cont = pd.DataFrame(data=X_cont, index=df.index, columns=col_names_cont)
df_cont.head()

,acqic,bacno,cano,conam,loctm,mcc,mchno,scity
txkey,,,,,,,,
516056,0.582079,0.658734,-1.162081,-0.341794,0.509681,2.028863,0.111604,-2.398364
4376,-3.987783,1.107708,-1.036278,-0.461282,-0.782179,1.952344,-1.812952,0.536031
483434,0.582079,-1.409013,1.297514,-0.341794,0.123412,2.028863,0.111604,-2.398364
1407164,0.472498,1.586350,-1.294169,0.903959,0.515305,-0.649316,-0.176983,-0.743258
1051004,-0.019621,0.504984,-0.453988,0.153869,0.690956,-0.445264,1.228783,0.536031


In [14]:
df_cont.describe()

,acqic,bacno,cano,conam,loctm,mcc,mchno,scity
count,1.943452e+06,1.943452e+06,1.943452e+06,1.943452e+06,1.943452e+06,1.943452e+06,1.943452e+06,1.943452e+06
mean,-5.220518e-15,3.877153e-16,-1.941865e-16,-1.708664e-14,-2.192115e-16,8.759717e-15,3.504502e-15,-5.581509e-14
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-3.987783e+00,-1.734602e+00,-1.784598e+00,-1.616042e+00,-2.792789e+00,-3.799365e+00,-1.812952e+00,-2.398364e+00
25%,-1.497170e-02,-8.620122e-01,-8.432618e-01,-5.495224e-01,-6.801126e-01,-6.110563e-01,-7.222740e-01,-4.047713e-01
50%,4.472611e-01,-2.565572e-03,1.465902e-02,-1.559142e-01,1.018168e-01,-4.325110e-01,1.116044e-01,5.360308e-01
75%,5.076967e-01,8.637698e-01,8.694213e-01,3.742951e-01,7.603139e-01,5.749946e-01,7.560210e-01,5.360308e-01
max,5.840714e-01,1.728499e+00,1.710698e+00,1.626205e+01,1.720611e+00,2.054370e+00,1.537969e+00,9.668325e-01


## 類別型

In [15]:
df_disc = df[col_names_disc].copy()
df_disc.head()

,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,ovrlt,stocn,stscd
txkey,,,,,,,,,,,,,
516056,5,0,N,0,N,N,5,N,0,33,N,102,0
4376,5,0,N,2,N,N,0,N,0,9,N,102,0
483434,5,0,N,0,N,N,5,N,0,6,N,102,0
1407164,5,62,N,5,N,N,5,N,0,5,N,102,0
1051004,5,62,N,4,N,N,5,N,0,6,N,102,0


In [16]:
df_disc.shape

(1943452, 13)

In [17]:
from sklearn.preprocessing import LabelEncoder
les = {}
for c in col_names_disc:
    le = LabelEncoder()
    df_disc.loc[:,c] = le.fit_transform(df_disc.loc[:,c])
    les.update({c:le})
df_disc.head()

,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,ovrlt,stocn,stscd
txkey,,,,,,,,,,,,,
516056,5,0,0,0,0,0,5,0,0,32,0,102,0
4376,5,0,0,2,0,0,0,0,0,8,0,102,0
483434,5,0,0,0,0,0,5,0,0,5,0,102,0
1407164,5,62,0,5,0,0,5,0,0,4,0,102,0
1051004,5,62,0,4,0,0,5,0,0,5,0,102,0


In [18]:
df_disc.describe()

,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,ovrlt,stocn,stscd
count,1.943452e+06,1.943452e+06,1.943452e+06,1.943452e+06,1.943452e+06,1.943452e+06,1.943452e+06,1.943452e+06,1.943452e+06,1.943452e+06,1.943452e+06,1.943452e+06,1.943452e+06
mean,4.825625e+00,5.378875e+01,2.566027e-01,4.142680e+00,1.496615e-02,9.537771e-02,4.748365e+00,2.708840e-02,4.841128e-02,5.739281e+01,1.292391e-02,9.562308e+01,2.475286e-02
std,6.557577e-01,2.076681e+01,4.367583e-01,2.402570e+00,1.460248e-01,4.162614e-01,1.114383e+00,1.623411e-01,3.615652e-01,3.409522e+01,1.129464e-01,1.896446e+01,2.211643e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.000000e+00,6.200000e+01,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,5.000000e+00,0.000000e+00,0.000000e+00,2.800000e+01,0.000000e+00,1.020000e+02,0.000000e+00
50%,5.000000e+00,6.200000e+01,0.000000e+00,4.000000e+00,0.000000e+00,0.000000e+00,5.000000e+00,0.000000e+00,0.000000e+00,5.700000e+01,0.000000e+00,1.020000e+02,0.000000e+00
75%,5.000000e+00,6.200000e+01,1.000000e+00,5.000000e+00,0.000000e+00,0.000000e+00,5.000000e+00,0.000000e+00,0.000000e+00,8.600000e+01,0.000000e+00,1.020000e+02,0.000000e+00
max,6.000000e+00,7.500000e+01,1.000000e+00,1.000000e+01,2.000000e+00,2.000000e+00,9.000000e+00,1.000000e+00,8.000000e+00,1.190000e+02,1.000000e+00,1.080000e+02,4.000000e+00


In [19]:
from sklearn.preprocessing import StandardScaler
scaler1 = StandardScaler()
X_disc = scaler.fit_transform(df_disc)
X_disc.shape, type(X_disc)

((1943452, 13), numpy.ndarray)

In [20]:
df_disc = pd.DataFrame(X_disc, index=df.index, columns=col_names_disc)
df_disc.head()

,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,ovrlt,stocn,stscd
txkey,,,,,,,,,,,,,
516056,0.265913,-2.590131,-0.587517,-1.724270,-0.10249,-0.229129,0.225806,-0.166861,-0.133894,-0.744762,-0.114425,0.336256,-0.111921
4376,0.265913,-2.590131,-0.587517,-0.891828,-0.10249,-0.229129,-4.260982,-0.166861,-0.133894,-1.448673,-0.114425,0.336256,-0.111921
483434,0.265913,-2.590131,-0.587517,-1.724270,-0.10249,-0.229129,0.225806,-0.166861,-0.133894,-1.536662,-0.114425,0.336256,-0.111921
1407164,0.265913,0.395402,-0.587517,0.356835,-0.10249,-0.229129,0.225806,-0.166861,-0.133894,-1.565992,-0.114425,0.336256,-0.111921
1051004,0.265913,0.395402,-0.587517,-0.059386,-0.10249,-0.229129,0.225806,-0.166861,-0.133894,-1.536662,-0.114425,0.336256,-0.111921


In [21]:
df_disc.describe()

,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,ovrlt,stocn,stscd
count,1.943452e+06,1.943452e+06,1.943452e+06,1.943452e+06,1.943452e+06,1.943452e+06,1.943452e+06,1.943452e+06,1.943452e+06,1.943452e+06,1.943452e+06,1.943452e+06,1.943452e+06
mean,-5.074577e-14,-4.369482e-13,-1.955514e-14,-2.048428e-14,8.673177e-14,-5.186651e-13,-1.475914e-13,-3.274769e-14,-3.476789e-14,-3.394101e-13,1.047055e-13,-1.244524e-13,7.268967e-14
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-7.358856e+00,-2.590131e+00,-5.875165e-01,-1.724270e+00,-1.024905e-01,-2.291294e-01,-4.260982e+00,-1.668610e-01,-1.338937e-01,-1.683310e+00,-1.144252e-01,-5.042226e+00,-1.119207e-01
25%,2.659135e-01,3.954024e-01,-5.875165e-01,-8.918283e-01,-1.024905e-01,-2.291294e-01,2.258063e-01,-1.668610e-01,-1.338937e-01,-8.620804e-01,-1.144252e-01,3.362561e-01,-1.119207e-01
50%,2.659135e-01,3.954024e-01,-5.875165e-01,-5.938626e-02,-1.024905e-01,-2.291294e-01,2.258063e-01,-1.668610e-01,-1.338937e-01,-1.152106e-02,-1.144252e-01,3.362561e-01,-1.119207e-01
75%,2.659135e-01,3.954024e-01,1.702080e+00,3.568348e-01,-1.024905e-01,-2.291294e-01,2.258063e-01,-1.668610e-01,-1.338937e-01,8.390383e-01,-1.144252e-01,3.362561e-01,-1.119207e-01
max,1.790867e+00,1.021401e+00,1.702080e+00,2.437940e+00,1.359381e+01,4.575545e+00,3.815237e+00,5.993010e+00,2.199213e+01,1.806916e+00,8.739334e+00,6.526375e-01,1.797419e+01


# One-Hot-Encoding

In [31]:
# from sklearn.preprocessing import OneHotEncoder
# ohe = OneHotEncoder(sparse=False)
# X_disc = ohe.fit_transform(df_disc)
# # cut_point = ohe.feature_indices_
# # print("feature cut point: ", cut_point)
# X_disc.shape, type(X_disc)

((1943452, 359), numpy.ndarray)

In [32]:
# new_col_names_disc = []
# for c in col_names_disc: 
#     le = les[c]
#     new_col_names_disc += [ c+'_'+str(cl) for cl in le.classes_ ]
# assert len(new_col_names_disc) == X_disc.shape[1]

In [33]:
# df_disc = pd.DataFrame(data=X_disc, index=df.index, columns=new_col_names_disc)
# df_disc.head()

,contp_0,contp_1,contp_2,contp_3,contp_4,contp_5,contp_6,csmcu_0,csmcu_1,csmcu_2,csmcu_3,csmcu_4,csmcu_5,csmcu_6,csmcu_7,csmcu_8,csmcu_9,csmcu_10,csmcu_11,csmcu_12,csmcu_13,csmcu_14,csmcu_15,csmcu_16,csmcu_17,csmcu_18,csmcu_19,csmcu_20,csmcu_21,csmcu_22,csmcu_23,csmcu_24,csmcu_25,csmcu_26,csmcu_27,csmcu_28,csmcu_29,csmcu_30,csmcu_31,csmcu_32,...,stocn_74,stocn_75,stocn_76,stocn_77,stocn_78,stocn_79,stocn_80,stocn_81,stocn_82,stocn_83,stocn_84,stocn_85,stocn_86,stocn_87,stocn_88,stocn_89,stocn_90,stocn_91,stocn_92,stocn_93,stocn_94,stocn_95,stocn_96,stocn_97,stocn_98,stocn_99,stocn_100,stocn_101,stocn_102,stocn_103,stocn_104,stocn_105,stocn_106,stocn_107,stocn_108,stscd_0,stscd_1,stscd_2,stscd_3,stscd_4
txkey,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
516056,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4376,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
483434,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1407164,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1051004,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [34]:
# df_disc.columns

Index(['contp_0', 'contp_1', 'contp_2', 'contp_3', 'contp_4', 'contp_5',
       'contp_6', 'csmcu_0', 'csmcu_1', 'csmcu_2',
       ...
       'stocn_104', 'stocn_105', 'stocn_106', 'stocn_107', 'stocn_108',
       'stscd_0', 'stscd_1', 'stscd_2', 'stscd_3', 'stscd_4'],
      dtype='object', length=359)

# Concatenate and save to .csv

In [22]:
df = pd.concat((df_cont, df_disc), 1)
df.shape

(1943452, 21)

In [23]:
# 取出 train.csv 的部份，還原 training set
df_X_train_origin = df.loc[y_train.index]
df_X_train_origin.shape

(1521787, 21)

In [24]:
df_X_test_origin = df.drop(labels=y_train.index, axis=0)
df_X_test_origin.shape

(421665, 21)

In [25]:
# 先取消索引
df_X_train_origin = df_X_train_origin.reset_index()
df_X_train_origin.shape

(1521787, 22)

In [26]:
# segment train & test
cut = 1217430
df_X_train = df_X_train_origin.loc[0:cut,:]
df_X_test = df_X_train_origin.loc[cut:,:]

In [27]:
# set index = 'txkey'
df_X_train = df_X_train.set_index("txkey")
df_X_test = df_X_test.set_index("txkey")

In [28]:
df_X_train.shape, df_X_test.shape

((1217431, 21), (304357, 21))

In [30]:
# df.to_csv("./Credit_Card/df_all_prep.csv")

In [31]:
df_X_test_origin.to_csv("test_origin_prep.csv")

In [32]:
df_X_train.to_csv("X_train.csv")

In [33]:
df_X_test.to_csv("X_test.csv")

In [34]:
y_train.to_csv("y_train.csv") # <Series>, index = 'txkey'

# 轉換 Testing 資料

In [38]:
df_test

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd
txkey,,,,,,,,,,,,,,,,,,,,,
592489,6881,163188,116799,513.80,5,0,N,0,N,N,5,N,0,102,215328.0,457,59360,N,0,102,0
592452,6881,163188,116799,513.80,5,0,N,0,N,N,5,N,0,102,222007.0,457,59360,N,0,102,0
590212,6881,163188,116799,513.80,5,0,N,0,N,N,5,N,0,100,170013.0,457,59360,N,0,102,0
590209,6881,163188,116799,513.80,5,0,N,0,N,N,5,N,0,100,165914.0,457,59360,N,0,102,0
592488,6881,163188,116799,513.80,5,0,N,0,N,N,5,N,0,102,215311.0,457,59360,N,0,102,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187507,5975,147326,79511,633.76,5,62,N,2,N,N,5,N,0,116,111228.0,343,87282,N,5817,102,0
1182598,6716,14305,136493,952.84,5,62,N,5,N,N,8,N,0,107,85839.0,245,48784,N,5859,102,0
898724,5975,156543,137963,713.42,5,62,N,4,N,N,5,N,0,111,184921.0,263,98326,N,5817,102,0
